# Imports & Definitions
```
$ pip freeze
tensorflow
scikit-image
scikit-learn
imutils
opencv-python
```

In [193]:
import tensorflow
from tensorflow import keras
import numpy
import skimage.io as scikit_io
import skimage.transform as scikit_transform
import matplotlib.pyplot as matplot_plot
import matplotlib.image as matplot_image
import sklearn.model_selection as scikit_model_selection
import os
import random
from imutils import paths

LEARNING_RATE = 0.0001
EPOCHS = 100
BATCH_SIZE = 10

INPUT_SIZE = 128
CHANNELS = 3

# Image Loading

In [194]:
        def load_image(path):
            image = scikit_io.imread(path)
            return scikit_transform.resize(image, (INPUT_SIZE, INPUT_SIZE), anti_aliasing = True)

        def show_image(image):
            matplot_plot.imshow(image)
            matplot_plot.axis('off')

In [195]:
datasource_dir = "../image-puller/images/"
dataset_dir = "images/"
labels = ["men", "women"]

random.seed(1)
test_train_ratio = 0.2

# Copy images from source into split directories of train / test for the given label
def prepare_directory(source, label):
    for image_path in os.listdir(source):
        if (image_path.endswith(".png")):
            image_set = "test" if random.random() < test_train_ratio else "train"
            destination = os.path.join(dataset_dir, image_set, label, image_path)
            scikit_io.imsave(destination, load_image(os.path.join(source, image_path)))
def prepare(label):
    prepare_directory(os.path.join(datasource_dir, label), label)
            

# Prepare Dataset

In [196]:
# Make the directories
for split_dir in ["train/", "test/"]:
    for label in labels:
        os.makedirs(dataset_dir + split_dir + label + "/", exist_ok = True)

# Copy the images over
prepare("men")
prepare("women")


# Compile the images into training, testing and validation sets

In [197]:
def compile(images):
    dataset = numpy.ndarray((len(images), INPUT_SIZE, INPUT_SIZE, CHANNELS), dtype = numpy.uint8)
    labels = numpy.ndarray((len(images)), dtype = numpy.uint8)
    for index, image in enumerate(images):
        image_data = scikit_io.imread(image)
        # Remove greyscale images
        if (len(image_data.shape) != 3):
            image_data = numpy.stack([image_data]*3, axis = -1)
        dataset[index] = image_data
        labels[index] = 0 if "women" in image else 1
    return dataset, labels


In [198]:
training_images = list(imutils.paths.list_images(dataset_dir + "train/"))
training_images_men = list(imutils.paths.list_images(dataset_dir + "train/men/"))
training_images_women = list(imutils.paths.list_images(dataset_dir + "train/women/"))
testing_images_men = list(imutils.paths.list_images(dataset_dir + "test/men/"))
testing_images_women = list(imutils.paths.list_images(dataset_dir + "test/women/"))
testing_images = training_images_men + training_images_women

In [199]:
TRAINING_SIZE_HALF = 10000
VALIDATION_SIZE_HALF = TRAINING_SIZE_HALF + 3000
training_data, training_labels = compile(training_images_men[:TRAINING_SIZE_HALF] + training_images_women[:TRAINING_SIZE_HALF])
validation_data, validation_labels = compile(training_images_men[TRAINING_SIZE_HALF:VALIDATION_SIZE_HALF] + training_images_women[TRAINING_SIZE_HALF:VALIDATION_SIZE_HALF])
testing_data, testing_images = compile(testing_images)

# Flatten the images into one numpy array

In [200]:
def create_model():
    model = keras.Sequential([
        keras.layers.Conv2D(16, (8, 8), input_shape = [128, 128, 3], strides = 1, padding = "same", activation = "relu", name = "convolution_large"),
        keras.layers.MaxPool2D(pool_size = (2, 2), padding = "valid", name = "pooling"),
        keras.layers.Conv2D(16, (4, 4), input_shape = [64, 64, 16], strides = 1, padding = "same", activation = "relu", name = "convolution_small"),
        keras.layers.Flatten(name = "flatten"),
        keras.layers.Dense(16 * 4 * 4, activation = tensorflow.nn.softmax, name = "fully_connected_1"),
        keras.layers.Dense(2, activation = tensorflow.nn.softmax, name = "fully_connected_2")
    ])
    model.compile(loss = "binary_crossentropy", optimizer = keras.optimizers.RMSprop(), metrics = [ "accuracy" ])
    return model


In [201]:
barnet = create_model()
barnet.summary()
callbacks = [
    keras.callbacks.History(),
    keras.callbacks.EarlyStopping(monitor = "loss", patience = 10, verbose = 1, mode = "auto")
]

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution_large (Conv2D)   (None, 128, 128, 16)      3088      
_________________________________________________________________
pooling (MaxPooling2D)       (None, 64, 64, 16)        0         
_________________________________________________________________
convolution_small (Conv2D)   (None, 64, 64, 16)        4112      
_________________________________________________________________
flatten (Flatten)            (None, 65536)             0         
_________________________________________________________________
fully_connected_1 (Dense)    (None, 256)               16777472  
_________________________________________________________________
fully_connected_2 (Dense)    (None, 2)                 514       
Total params: 16,785,186
Trainable params: 16,785,186
Non-trainable params: 0
_________________________________________

# TRAIN!

In [203]:
barnet.fit(training_data, training_labels, batch_size = BATCH_SIZE, epochs = 20, validation_data = (validation_data, validation_labels), verbose = 2, shuffle = True, callbacks = callbacks)

Epoch 1/20
2000/2000 - 304s - loss: 0.6932 - accuracy: 0.5032 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/20
2000/2000 - 299s - loss: 0.6932 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/20
2000/2000 - 299s - loss: 0.6932 - accuracy: 0.4975 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/20
2000/2000 - 304s - loss: 0.6932 - accuracy: 0.5007 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/20
2000/2000 - 303s - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/20
2000/2000 - 299s - loss: 0.6932 - accuracy: 0.4970 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/20
2000/2000 - 300s - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/20
2000/2000 - 305s - loss: 0.6932 - accuracy: 0.5061 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 9/20
2000/2000 - 302s - loss: 0.6932 - accuracy: 0.5033 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 10/20
2000/2000 - 300s - loss: 0.6932 - accuracy: 0.4973 -

In [204]:
test_loss, test_accuracy = vgg.evaluate(validation_data, validation_labels, verbose = 1)
print(f"Loss: {test_loss}\nAccuracy: {test_accuracy}")

188/188 [==============================] - 0s 659us/step - loss: 0.0000e+00


TypeError: cannot unpack non-iterable float object